In [8]:
from bs4 import BeautifulSoup
import re
import chardet
import pandas as pd
import numpy as np
import time
import os
from datetime import datetime
from db_conn.connection import *
from IPython.display import clear_output

In [34]:
def scrape_html() -> list[pd.DataFrame]:
    global today
    today = datetime.now().strftime('%m%d%Y')
    rows = []
    files = [x for x in os.listdir(f'./html_source/subchapters/{today}') if x != '.DS_Store']
    cnt = 0
    total= len(files)
    for x in files:
        clear_output(wait=True)
        cnt += 1
        print(f'file: {cnt} out of {total}....{x}')
        with open(f'./html_source/subchapters/{today}/{x}', 'rb') as f:
            soup = BeautifulSoup(f, 'lxml')
        _title = soup.find('title').text.strip() if soup.find('title') else None
        _heading= None
        _body = None
        for y in soup.find_all('p'):
            if 'heading' in y.get('class')[0].lower():
                _heading = y.text.strip()
                _body = None
            if 'body' in y.get('class')[0].lower():
                _body = y.text
            row = pd.DataFrame({
                    'file_name':    [x],
                    'title':        [_title],
                    'heading':      [_heading],
                    'body_text':    [_body]
                })
            rows.append(row)
    return rows


In [20]:
rows = scrape_html()

file: 1500 out of 1500....03FAH012110.html


In [21]:
df = pd.concat(rows, ignore_index=True)

In [22]:
df

,file_name,title,heading,body_text
0,04FAM0830.html,4 FAM 830 EMERGENCY EVACUATION FISCAL POLICY,None,None
1,04FAM0830.html,4 FAM 830 EMERGENCY EVACUATION FISCAL POLICY,4 FAM 830,None
2,04FAM0830.html,4 FAM 830 EMERGENCY EVACUATION FISCAL POLICY,EMERGENCY EVACUATION,None
3,04FAM0830.html,4 FAM 830 EMERGENCY EVACUATION FISCAL POLICY,FISCAL POLICY,None
4,04FAM0830.html,4 FAM 830 EMERGENCY EVACUATION FISCAL POLICY,FISCAL POLICY,None
...,...,...,...,...
253003,03FAH012110.html,3 FAH-1 H-2110 DRUG-FREE WORKPLACE,3 FAH-1 Exhibit H-2111\n \n\n U.S. Depar...,\n\n XV. LIST OF POSITIONS DESIGNATED FO...
253004,03FAH012110.html,3 FAH-1 H-2110 DRUG-FREE WORKPLACE,3 FAH-1 Exhibit H-2111\n \n\n U.S. Depar...,\n\n XV. LIST OF POSITIONS DESIGNATED FO...
253005,03FAH012110.html,3 FAH-1 H-2110 DRUG-FREE WORKPLACE,3 FAH-1 Exhibit H-2111\n \n\n U.S. Depar...,\n\n XV. LIST OF POSITIONS DESIGNATED FO...
253006,03FAH012110.html,3 FAH-1 H-2110 DRUG-FREE WORKPLACE,3 FAH-1 Exhibit H-2111\n \n\n U.S. Depar...,\n A complete list of all TDP positions by ...


In [35]:
df.to_sql('fam_{today}', engine(), if_exists='replace', index=False)

8

In [36]:
df_g = (
    df.loc[df['body_text'].notna(), :]
    .reset_index(drop=True)
    .groupby(['file_name', 'title', 'heading'], as_index = False)['body_text']
    .apply(lambda x: ' \n '.join(x))
)

In [37]:
df_g['heading'] = df_g['heading'].str.split('\n')

In [38]:
df_g['ref'] = df_g['heading'].apply(lambda x: ''.join([i for i in x if 'fam' in i.lower()]) if x else None)
df_g['heading'] = df_g['heading'].apply(lambda x: ''.join([i for i in x if 'fam' not in i.lower()]) if x else None).str.upper().str.strip()

In [39]:
df_g = df_g.replace('\n', '')

In [40]:
df_g.to_sql('fam_grouped_{today}', engine(), if_exists='replace', index=False)

852